# Workbook for optimizing foci segmentation thresholds

This workbook provides step-by-step instructions for determining segmentation thresholds for identifying foci within cells. __Do not edit this workbook file directly!__ Make a copy for each optimization that you perform and edit that version. Save this template in an unedited form for future experiments.

This workbook uses a built-in "optimization mode" that only pulls the first three images out of multi-image .czi files to dramatically speed up processing. This will allow you to tinker with thresholds and find the ideal cutoffs.

I found that the most effective way to define these cutoffs was to use a positive control image - one where there were many foci - and a negative control image with few to no foci to set the thresholds. This allows you to identify cutoffs that simultaneously maximize sensitivity while minimizing false positives.

__Before running this notebook, you must have completed the setup notebook in the install folder.__

In [2]:
# import required code
import sys
sys.path.append('/Users/nweir/Dropbox/code/csth-imaging')
from csth_analysis import find_cells, segment_cells, foci
import os
# uncomment the following lines if you have jupyternotify installed and want to use it
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

/usr/local/lib/python3.6/site-packages/tifffile.py:313: UserWarning: ImportError: No module named '_tifffile'. Loading of some compressed images will be very slow. Tifffile.c can be obtained at http://www.lfd.uci.edu/~gohlke/
  "ImportError: No module named '_tifffile'. "
/usr/local/lib/python3.6/site-packages/czifile.py:130: UserWarning: failed to import the optional _czifile C extension module.
Decoding of JXR and JPEG encoded images will not be available.
Czifile.pyx can be obtained at http://www.lfd.uci.edu/~gohlke/
  "failed to import the optional _czifile C extension module.\n"
/usr/local/lib/python3.6/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


<IPython.core.display.Javascript object>

The cell below defines the file paths for the files you will need during this process. Change each variable as needed.

In [3]:
dATG7_czi = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/Lysoview-LC3_HEK_dAtg7_3hrTor_9Pos_1_AiryscanProcessing.czi'
dATG7_empty = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/Lysoview-LC3_HEK_dAtg7_3hrTor_Empty_AiryscanProcessing.czi'
dFIP200_czi = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/Lysoview-LC3_HEK_dFIP200_3hrTor_7Pos_AiryscanProcessing.czi'
dFIP200_empty = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/Lysoview-LC3_HEK_dFIP200_3hrTor_Empty_AiryscanProcessing.czi'
WT_czi = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/Lysoview-LC3_HEK_WT_3hrTor_6Pos_AiryscanProcessing.czi'
WT_empty = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/Lysoview-LC3_HEK_WT_3hrTor_Empty_AiryscanProcessing.czi'
dTMEM_czi = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/Lysoview-LC3_HEK_dTMEM_3hrTor_5Pos_AiryscanProcessing.czi'
dTMEM_empty = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/Lysoview-LC3_HEK_dTMEM_3hrTor_Empty_AiryscanProcessing.czi'
dVPS37A_czi = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/Lysoview-LC3_HEK_dVPS37A_3hrTor_10Pos_1_AiryscanProcessing.czi'
dVPS37A_empty = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/Lysoview-LC3_HEK_dVPS37A_3hrTor_Empty_AiryscanProcessing.czi'
svm_pkl = '/Users/nweir/Dropbox/code/csth-imaging/trained_svm.pkl' # change /path/to/ so it points to the csth-imaging folder

Now that you have defined the important variables, you will perform pre-processing. This takes the following steps:
1. Load images
2. Identify regions of the field containing cells
3. Identify nuclei
4. Segment cells  
    _Note:_ By default this segments cells using the 488 wavelength. Change 488 to your desired wavelength in the splitter.segment_cells() command to change wavelengths.
5. Initialize foci detection object.

The steps are indicated in the code blocks.

In [4]:
%%notify
## NOTE: RUNNING THIS CELL WILL TAKE A WHILE! DON'T BE CONCERNED! ##

# 1. load images and the trained SVM focus classifier, and 2.
dVPS37A_finder = find_cells.MultiFinder(filename=dVPS37A_czi,
                                         bg_filename=dVPS37A_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
dATG7_finder = find_cells.MultiFinder(filename=dATG7_czi,
                                         bg_filename=dATG7_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
dTMEM_finder = find_cells.MultiFinder(filename=dTMEM_czi,
                                         bg_filename=dTMEM_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
dFIP200_finder = find_cells.MultiFinder(filename=dFIP200_czi,
                                         bg_filename=dFIP200_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
WT_finder = find_cells.MultiFinder(filename=WT_czi,
                                         bg_filename=WT_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
print('MultiFinder created.')

# 2. find cells
WT_splitter = segment_cells.CellSplitter(WT_finder)
dVPS37A_splitter = segment_cells.CellSplitter(dVPS37A_finder)
dATG7_splitter = segment_cells.CellSplitter(dATG7_finder)
dTMEM_splitter = segment_cells.CellSplitter(dTMEM_finder)
dFIP200_splitter = segment_cells.CellSplitter(dFIP200_finder)

# 3. identify nuclei
WT_splitter.segment_nuclei(verbose=True)
dVPS37A_splitter.segment_nuclei(verbose=True)
dATG7_splitter.segment_nuclei(verbose=True)
dTMEM_splitter.segment_nuclei(verbose=True)
dFIP200_splitter.segment_nuclei(verbose=True)

# 4. Segment cells.
WT_splitter.segment_cells(488, verbose=True)
dVPS37A_splitter.segment_cells(488, verbose=True)
dATG7_splitter.segment_cells(488, verbose=True)
dTMEM_splitter.segment_cells(488, verbose=True)
dFIP200_splitter.segment_cells(488, verbose=True)

# 5. Initialize foci detection objects.
dVPS37A_foci = foci.Foci(dVPS37A_splitter, verbose=True)
dATG7_foci = foci.Foci(dATG7_splitter, verbose=True)
dFIP200_foci = foci.Foci(dFIP200_splitter, verbose=True)
dTMEM_foci = foci.Foci(dTMEM_splitter, verbose=True)
WT_foci = foci.Foci(WT_splitter, verbose=True)

Optimization mode: only using first three images.
Optimization mode: only using first three images.
Optimization mode: only using first three images.
Optimization mode: only using first three images.
Optimization mode: only using first three images.
MultiFinder created.
generating cell masks...
log-transforming arrays...
applying gaussian filter...
computing p-value transformation...
 computing p-val xform for image 1 out of 3
 computing p-val xform for image 2 out of 3
 computing p-val xform for image 3 out of 3
converting to binary...

generating cell masks...

generating mask #1
labeling contiguous objects...
eliminating objects w/volume < 100,000 px...
pruning labels...
unlabeling out of focus slices...
generating gradient image...
generating gradient intensity histograms...
predicting focus labels using SVM classifier...
calculating decision function values for labels...
raw focus labels:
[0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1

eliminating objects w/volume < 100,000 px...
pruning labels...
unlabeling out of focus slices...
generating gradient image...
generating gradient intensity histograms...
predicting focus labels using SVM classifier...
calculating decision function values for labels...
raw focus labels:
[0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
correcting slice labels...
[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
appending outputs...
mask #3 complete.

generating cell masks...
log-transforming arrays...
applying gaussian filter...
computing p-value transformation...
 computing p-val xform for image 1 out of 3
 computing p-val xform for image 2 out of 3
 computing p-val xform for image 3 out of 3
converting to binary...

generating cell masks...

generating mask #1
labeling cont

creating PexSegmentObj...
passing segmented objs and seeds to Nuclei instance...

segmenting image #3 of 3...
Using thresholds set based on slice intensity
thresholding floor set to 3156
slice-by-slice cutoffs:
[   71.     50.5    66.     78.5    83.     93.5    98.    120.5   127.
   134.    154.    173.    238.5   278.5   334.5   373.    414.5   516.5
   810.   1438.5  2387.5  3467.   4421.   5056.5  6008.5  6770.5  7165.5
  7494.   7632.   7564.   7490.5  7533.   7542.5  7765.5  7891.5  7814.
  7580.   7210.5  7065.   6909.   6819.   6506.5  6536.   6453.   5991.5
  5056.5  3849.   2709.   1940.   1517.5  1251.5]
generating threshold image...
raw image imported.
preprocessing complete.
filling holes in objects.
holes filled.
distance map complete.
smoothing distance map...
distance map smoothed.
identifying maxima...
maxima identified.
watershedding...
watershedding complete.
creating PexSegmentObj...
passing segmented objs and seeds to Nuclei instance...

removing small perinuclear

raw image imported.
preprocessing complete.
filling holes in objects.
holes filled.
distance map complete.
smoothing distance map...
distance map smoothed.
identifying maxima...
maxima identified.
watershedding...
watershedding complete.
creating PexSegmentObj...
passing segmented objs and seeds to Nuclei instance...

segmenting image #2 of 3...
Using thresholds set based on slice intensity
thresholding floor set to 2747
slice-by-slice cutoffs:
[  385.5   458.5   692.5  1090.   1726.   2498.5  3417.   4369.5  5262.
  6011.   6437.   6554.   6725.5  6861.5  6716.   6546.5  6462.5  6431.5
  6399.   6438.   6710.   6868.   6759.5  6670.   6597.5  6631.5  6650.
  6540.   6409.5  6313.   6286.5  5980.   5665.   5679.   5500.   5277.5
  5240.   5311.   5437.5  5517.   5368.5  4969.   4801.5  4760.5  4686.5
  4480.5  4189.   3741.   3212.   2568.5  1977.5]
generating threshold image...
raw image imported.
preprocessing complete.
filling holes in objects.
holes filled.
distance map complete.
s

/usr/local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


cell mask means:
[1840.922071317907, 1900.3607769726525, 1949.7208244053584, 1987.6069771551529, 2014.8685812066099, 2033.6259341591795, 2046.1479818612224, 2054.2321101771145, 2061.6092332560129, 2069.9763302765482]
slopes:
[54.39937654372568, 43.623100091250194, 32.573878400625745, 23.00947850201328, 15.639700327306286, 10.303088008967507, 7.7306256973952259, 7.8721100497168663]
slope deltas:
[ 10.77627645  11.04922169   9.5643999    7.36977817   5.33661232
   2.57246231   0.14148435]
desired erosions: 3
erosion of mask #0 complete.
eroding mask 1...
cell mask means:
[1718.9112187047224, 1797.9503440118463, 1865.6569165375515, 1917.7236503468887, 1952.0165403486794, 1969.1109528932768, 1973.801440598816, 1971.2872974322765, 1964.6228548623121, 1954.2625695919187]
slopes:
[73.372848916414569, 59.886653167521217, 43.179811905563952, 25.693651273194064, 10.892450125068308, 1.0881722694998643, -4.5892928682519596, -8.5123639201789274]
slope deltas:
[ 13.48619575  16.70684126  17.48616063

watershed segmenting cells from nuclei centers...
segmentation complete for image #3.
removing cells that contact the edge of the image.
removing small foci...
2 raw segmented cells
0 foci volume < 1000 px, removing...

checking that nuclei have already been segmented...
eroding cell edges...
eroding mask 0...
cell mask means:
[2975.9990421450148, 3089.5685090137858, 3197.8512624138957, 3295.0147952202306, 3375.5625353295745, 3437.5028298182406, 3482.0934005409395, 3510.6967636599493, 3528.8320429149062, 3542.4154763591882]
slopes:
[110.92611013444048, 102.72314310322236, 88.85563645783941, 71.244017299005009, 53.265432605682463, 36.596966920854356, 23.36932118698337, 15.859356349619475]
slope deltas:
[  8.20296703  13.86750665  17.61161916  17.97858469  16.66846568
  13.22764573   7.50996484]
desired erosions: 5
erosion of mask #0 complete.
eroding mask 1...
cell mask means:
[2456.3503693933731, 2527.9592400039433, 2594.107501519019, 2653.7903824561959, 2706.6990899995744, 2752.877722

<IPython.core.display.Javascript object>

Next, you're ready to try segmenting foci and checking the output to see if it looks correct.

A bit of background on how Canny edge detection works so you can logically select values. There are two threshold numbers that go into detecting foci:
- high threshold: the minimum edge intensity to _seed_ a new object. The entire object doesn't have to have an edge this sharp, but if no part of it does, it won't be identified.
- low threshold: the minimum edge intensity to _grow_ an object. If one point already satisfied the high threshold, it can grow even if it has a dimmer edge as long as that dimmer edge is above the low threshold.  

If this doesn't make sense, read about the Canny edge detection algorithm on Wikipedia or elsewhere.

I often start with values of 10,000 and 7,500 for the high and low thresholds respectively, so those are the default values below.

Running the code block below will perform segmentation and save the raw images as well as the segmented foci to the output_dir you select in the first line. You can then open these images in fiji, overlay the segmented foci onto the raw images, and determine if the threshold is too high/too low. The code will also output some descriptive information about the number of dots, their intensities, etc. which may be useful to you in deciding thresholds.

In [7]:
from skimage import io  # for saving images
output_dir = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputs/'  # change this to where you'd like the output images to go.
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
# save raw images
for c in dATG7_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, dATG7_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'dAtg7_raw_' + str(c) + '_' + str(im) + '.tif',
                 dATG7_foci.imgs[c][im, :, :, :])
for c in dVPS37A_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, dVPS37A_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'dVPS37A_raw_' + str(c) + '_' + str(im) + '.tif',
                 dVPS37A_foci.imgs[c][im, :, :, :])    
for c in dTMEM_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, dTMEM_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'dTMEM_raw_' + str(c) + '_' + str(im) + '.tif',
                 dTMEM_foci.imgs[c][im, :, :, :])
for c in dFIP200_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, dFIP200_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'dFIP200_raw_' + str(c) + '_' + str(im) + '.tif',
                 dFIP200_foci.imgs[c][im, :, :, :])
for c in WT_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, WT_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'WT_raw_' + str(c) + '_' + str(im) + '.tif',
                 WT_foci.imgs[c][im, :, :, :])

/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputsdAtg7_raw_488_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputsdAtg7_raw_488_1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputsdAtg7_raw_488_2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputsdAtg7_raw_561_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarnin

## The following cell is for running segmentation on both channels.

In [1]:
output_dir = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputs/'
# edit the (high, low) threshold pairs below to optimize segmentation.
green_thresholds = (10000, 7500) 
red_thresholds = (10000, 7500)
# perform segmentation
dATG7_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds})
for c in dATG7_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(dATG7_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'dATG7_foci_' + str(c) + '_' + str(im) + '.tif',
                 dATG7_foci.foci[c][im])
dVPS37A_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds})
for c in dVPS37A_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(dVPS37A_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'dVPS37A_foci_' + str(c) + '_' + str(im) + '.tif',
                 dVPS37A_foci.foci[c][im])
dTMEM_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds})
for c in dTMEM_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(dTMEM_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'dTMEM_foci_' + str(c) + '_' + str(im) + '.tif',
                 dTMEM_foci.foci[c][im])
dFIP200_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds})
for c in dFIP200_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(dFIP200_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'dFIP200_foci_' + str(c) + '_' + str(im) + '.tif',
                 dFIP200_foci.foci[c][im])
WT_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds})
for c in WT_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(WT_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'WT_foci_' + str(c) + '_' + str(im) + '.tif',
                 WT_foci.foci[c][im])

NameError: name 'dATG7_foci' is not defined

Edit the thresholds in the cell above and re-run as needed until you reach ideal thresholds.

## The following cell is for running segmentation on just one channel.

In [ ]:
desired_channel = 488  # change this to 488 for green or 561 for red
# edit the (high, low) threshold pair below to optimize segmentation.
thresholds = (10000, 7500) 
# perform segmentation
neg_foci.segment(verbose=True, thresholds={desired_channel: thresholds},
                 seg_channels=(desired_channel,))
pos_foci.segment(verbose=True, thresholds={desired_channel: thresholds},
                 seg_channels=(desired_channel,))
# save segmentation outputs
for im in range(0, len(pos_foci.foci[desired_channel])):  # for each image in that channel
    io.imsave(output_dir + 'pos_foci_' + str(desired_channel) + '_' + str(im) + '.tif',
             pos_foci.foci[desired_channel][im])
for im in range(0, len(neg_foci.foci[desired_channel])):  # for each image in that channel
    io.imsave(output_dir + 'neg_foci_' + str(desired_channel) + '_' + str(im) + '.tif',
             neg_foci.foci[desired_channel][im])